In [1]:
import sys
sys.path.append('../src')
from util import load_balls, preprocess_data
from dataset import nballDataset
from model import nballBertNorm, nballBertDirection

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch.nn.functional as F

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\Anaconda\envs\annotated-transformer\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [2]:
# Load data
train_semcor_path = {
    "xml":'../data/WSD_Evaluation_Framework/Training_Corpora/Semcor/semcor.data.xml',
    "gold_key":'../data/WSD_Evaluation_Framework/Training_Corpora/Semcor/semcor.gold.key.txt',}

eval_all_path = {
    "xml":'../data/WSD_Evaluation_Framework/Evaluation_Datasets/ALL/ALL.data.xml',
    "gold_key":'../data/WSD_Evaluation_Framework/Evaluation_Datasets/ALL/ALL.gold.key.txt',}

nball_path = '../data/sample_mammal/nball.txt'

nball = load_balls(nball_path)
train_semcor, lemma_pair = preprocess_data(train_semcor_path, nball)

loading balls....
388 balls are loaded



In [3]:
# Problem about lemma
display(train_semcor.head())
print(len(lemma_pair))
print(len(nball.keys()))

,lemma,word,sentence_text,lemma_idx,formatted_sense_id,sense_idx,sense_group
0,man,men,Whereas the eighteenth century had been a time...,131,homo.n.02,132,[132]
1,horse,horse,One wrote : `` [ I am so hungry ] I could eat ...,255,horse.n.01,261,[261]
2,wildcat,wildcat,"She is well-educated and refined , all wildcat...",53,wildcat.n.03,54,[54]
3,lion,Lions,"To find out , we traveled throughout that part...",61,lion.n.01,62,[62]
4,elephant,elephants,The Prince visited the hospital of Operation B...,103,elephant.n.01,104,[104]


375
388


In [4]:
# Loading nball embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sense_labels = list(nball.keys())
nball_embeddings = [nball[label].center for label in sense_labels]
nball_embeddings = torch.tensor(np.array(nball_embeddings), dtype=torch.float32).to(device)
nball_norms = [nball[label].distance for label in sense_labels]
nball_norms = torch.tensor(np.array(nball_norms), dtype=torch.float32).to(device)

In [5]:
# Model list
bert_models = {"BERT-Base": ["bert-base-uncased", 768], #768, 12L, 12A
               "BERT-Large": ["bert-large-uncased", 1024], #1024, 24L, 16A
               "BERT-Medium": ["google/bert_uncased_L-8_H-512_A-8", 512], #512, 8L, 8A
               "BERT-Small": ["google/bert_uncased_L-4_H-256_A-4", 256], #256, 4L, 4A
               "BERT-Mini": ["google/bert_uncased_L-4_H-128_A-2", 128],#128, 4L, 2A
               "BERT-Tiny": ["google/bert_uncased_L-2_H-128_A-2", 128]}#128, 2L, 2A

max_length = 512
batch_size = 32
model_url = bert_models["BERT-Small"][0]

In [6]:
dataset_train = nballDataset(train_semcor, nball, model_url, max_length)

Tokenizing sentences...
Tokenizing finished.
Calculating word indices...
Tokenizing finished.


In [7]:
dataloader_train = DataLoader(dataset_train, batch_size, shuffle=True)

In [8]:
def train(model, dataloader, optimizer, scheduler, num_epochs, mode):
    last_loss = 0
    if mode == "norm":
        loss_fn = log_cosh_loss
    elif mode == "direction":
        loss_fn = nn.CosineEmbeddingLoss()
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True, position=0)
        for batch in progress_bar:
            # Unpack batch and send to device
            batch_input_ids, batch_attention_masks, batch_word_indices, \
            batch_sense_indices, _, _ = [b.to(device) for b in batch]

            optimizer.zero_grad()
        
            # Forward pass
            output = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, \
                            word_index=batch_word_indices).squeeze()

            # print(output)
            if mode == "norm":
                batch_norms = nball_norms[batch_sense_indices]
                loss = loss_fn(output, batch_norms * 1e-5)
            elif mode == "direction":
                labels = torch.ones(output.size(0), device=device)
                batch_senses = nball_embeddings[batch_sense_indices]
                loss = loss_fn(output, batch_senses, labels)
                
            loss.backward()
            optimizer.step()
        
            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        # Logging average metrics per epoch
        avg_loss = total_loss / len(dataloader)
        improvement = (last_loss - total_loss) / len(dataloader) if last_loss != 0 else 0
        print(f'Epoch {epoch + 1}, Avg Loss: {avg_loss}, Improvement: {improvement}')
        last_loss = total_loss

        scheduler.step()

In [9]:
# Train norm
def log_cosh_loss(norm_u, norm_v):
    return torch.log(torch.cosh(norm_u - norm_v)).mean()

output_dim = 160
model_norm = nballBertNorm(model_url, output_dim).to(device)
loss_fn_norm = log_cosh_loss
optimizer_norm = optim.Adam([
    {'params': model_norm.parameters()}
], lr=2e-5)
scheduler_norm = StepLR(optimizer_norm, step_size=50, gamma=0.1)

train(model=model_norm, \
      dataloader=dataloader_train,\
      optimizer=optimizer_norm,\
      scheduler=scheduler_norm,\
      num_epochs=50, \
      mode="norm")

Epoch 1/50: 100%|███████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  5.65it/s, loss=6.61]


Epoch 1, Avg Loss: 6.756277517838911, Improvement: 0


Epoch 2/50: 100%|███████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 13.90it/s, loss=6.92]


Epoch 2, Avg Loss: 6.2669667330655185, Improvement: 0.4893107847733931


Epoch 3/50: 100%|███████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 13.94it/s, loss=5.65]


...

Epoch 48, Avg Loss: 0.9956685385920785, Improvement: 0.09901977940039201


Epoch 49/50: 100%|█████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 14.27it/s, loss=0.407]


Epoch 49, Avg Loss: 0.9077390784567053, Improvement: 0.0879294601353732


Epoch 50/50: 100%|█████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 14.52it/s, loss=0.179]

Epoch 50, Avg Loss: 0.8224796002561395, Improvement: 0.08525947820056569


In [10]:
# Train direction
model_d = nballBertDirection(model_url, output_dim).to(device)
optimizer_d = optim.Adam([
    {'params': model_d.parameters()}
], lr=2e-5)
scheduler_d = StepLR(optimizer_d, step_size=50, gamma=0.1)

train(model=model_d, \
      dataloader=dataloader_train,\
      optimizer=optimizer_d,\
      scheduler=scheduler_d,\
      num_epochs=20, \
      mode="direction")

Epoch 1/20: 100%|██████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 13.87it/s, loss=0.762]


Epoch 1, Avg Loss: 0.9002834450114857, Improvement: 0


Epoch 2/20: 100%|██████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 14.08it/s, loss=0.519]


Epoch 2, Avg Loss: 0.6333872188221324, Improvement: 0.26689622618935327


Epoch 3/20: 100%|██████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 14.37it/s, loss=0.346]


...

Epoch 18, Avg Loss: 0.020147095519033344, Improvement: 0.0014364707537672737


Epoch 19/20: 100%|████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 13.90it/s, loss=0.0194]


Epoch 19, Avg Loss: 0.019093484363772652, Improvement: 0.0010536111552606928


Epoch 20/20: 100%|████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 13.89it/s, loss=0.0187]

Epoch 20, Avg Loss: 0.018734653903679413, Improvement: 0.0003588304600932381


In [11]:
# Evaluation

def evaluation(model_n, model_d, dataloader):
    model_n.eval()
    model_d.eval()

    loss_fn_n = log_cosh_loss
    loss_fn_d = nn.CosineEmbeddingLoss()

    loss_n = 0.0
    loss_d = 0.0
    accuracy = 0.0
    correct_predictions = 0
    pred_indices = {}
    size = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluation", leave=True, position=0):
            batch_input_ids, batch_attention_masks, batch_word_indices, \
            batch_sense_indices, batch_lemma_indices, batch_idices = [b.to(device) for b in batch]

            # Forward pass
            norms = model_n(input_ids=batch_input_ids, attention_mask=batch_attention_masks, \
                            word_index=batch_word_indices).squeeze()
            directions = model_d(input_ids=batch_input_ids, attention_mask=batch_attention_masks, \
                                 word_index=batch_word_indices).squeeze()

            target_norms = nball_norms[batch_sense_indices]
            target_directions = nball_embeddings[batch_sense_indices]

            labels = torch.ones(directions.size(0), device=device)
            loss_n_batch = loss_fn_n(norms, target_norms * 1e-5)
            loss_d_batch = loss_fn_d(directions, target_directions, labels)
            loss_n += loss_n_batch.item()
            loss_d += loss_d_batch.item()

            # Predict
            # Get candidate group norms and directions based on lemma_pair mapping
            candidate_indices = [lemma_pair[l.item()] for l in batch_lemma_indices]
            candidate_norms = [nball_norms[idx] * 1e-5 for idx in candidate_indices]
            candidate_directions = [nball_embeddings[idx] for idx in candidate_indices]
            output_embeddings = directions * norms.unsqueeze(-1)
            # Calculate distances and predictions
            for i in range(len(output_embeddings)):
                size += 1
                output_emb = output_embeddings[i]
                distances = [F.pairwise_distance(output_emb.unsqueeze(0), 
                                                  (cand_dir * cand_norm.unsqueeze(-1)).unsqueeze(0)).item()
                             for cand_dir, cand_norm in zip(candidate_directions[i], candidate_norms[i])]
                min_distance_idx = distances.index(min(distances))
                predicted_index = candidate_indices[i][min_distance_idx]
                pred_indices[batch_idices[i].item()] = predicted_index
                if predicted_index == batch_sense_indices[i].item():
                    correct_predictions += 1
                

        total_batches = len(dataloader)
        loss_n /= total_batches
        loss_d /= total_batches
        accuracy = correct_predictions / size

    return loss_n, loss_d, accuracy, pred_indices


In [21]:
loss_n, loss_d, accurancy, pred_indices = evaluation(model_n=model_norm, \
                                                     model_d=model_d,\
                                                     dataloader=dataloader_train)

print(f"Average loss of norm:{loss_n}, average loss of dorection:{loss_d}")
print(f"Accurancy:{accurancy*100}%")
train_semcor['pred_sense_idx'] = train_semcor.index.map(pred_indices)
display(train_semcor.head())

Evaluation: 100%|██████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 17.10it/s]


Average loss of norm:0.656003089113669, average loss of dorection:0.0026112972674044695
Accurancy:99.43181818181817%


,lemma,word,sentence_text,lemma_idx,formatted_sense_id,sense_idx,sense_group,pred_sense_id,pred_sense_idx
0,man,men,Whereas the eighteenth century had been a time...,131,homo.n.02,132,[132],132,132
1,horse,horse,One wrote : `` [ I am so hungry ] I could eat ...,255,horse.n.01,261,[261],261,261
2,wildcat,wildcat,"She is well-educated and refined , all wildcat...",53,wildcat.n.03,54,[54],54,54
3,lion,Lions,"To find out , we traveled throughout that part...",61,lion.n.01,62,[62],62,62
4,elephant,elephants,The Prince visited the hospital of Operation B...,103,elephant.n.01,104,[104],104,104


In [22]:
count_single_element = (train_semcor['sense_group'].apply(lambda x: len(x) == 1)).sum()
mismatch_count = (train_semcor['sense_idx'] != train_semcor['pred_sense_idx']).sum()
count_multiple_element = set_length-count_single_element

set_length = len(train_semcor)
print(f"Length of dataset:{set_length}")
print(f"Length of have only one element:{count_single_element}")
print(f"Length of mismatch:{mismatch_count}")
print(f"Original accurancy:{count_single_element / set_length *100}%")
print(f"Filtered accurancy:{(count_multiple_element-mismatch_count) /count_multiple_element  *100}%")


Length of dataset:352
Length of have only one element:330
Length of mismatch:2
Original accurancy:93.75%
Filtered accurancy:90.9090909090909%


In [19]:
eval_all, lemma_pair = preprocess_data(eval_all_path, nball)
dataset_eval = nballDataset(eval_all, nball, model_url, max_length)
dataloader_eval = DataLoader(dataset_eval, batch_size, shuffle=True)
loss_n, loss_d, accurancy, pred_indices = evaluation(model_n=model_norm, \
                                                     model_d=model_d,\
                                                     dataloader=dataloader_eval)

eval_all['pred_sense_idx'] = eval_all.index.map(pred_indices)
print(f"Average loss of norm:{loss_n}, average loss of dorection:{loss_d}")
print(f"Accurancy:{accurancy}")
display(eval_all.head())

Tokenizing sentences...
Tokenizing finished.
Calculating word indices...
Tokenizing finished.


Evaluation: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.37it/s]

Average loss of norm:0.5518102645874023, average loss of dorection:0.0034370184876024723
Accurancy:1.0


,lemma,word,sentence_text,lemma_idx,formatted_sense_id,sense_idx,sense_group,pred_sense_idx
0,mouse,mice,His doubts stemmed from the fact that several ...,183,mouse.n.01,184,[184],184
1,cat,cats,"The cats are fine , although nervous .",54,cat.n.01,55,[55],55
2,mouse,mouse,The quivers move through my house every few mi...,183,mouse.n.01,184,[184],184
3,cow,cows,After long stretches of this attendant ground ...,99,cow.n.01,310,"[100, 310]",310
4,world,world,But it was seen as an important advance in a n...,130,world.n.08,131,[131],131
